### Init

In [55]:
import robotic as ry
import time

C = ry.Config()
C.addFile("throwing1.g")
qHome = C.getJointState()
C.view()
#time.sleep(30)
C.getFrame('cargo').unLink()
C.view()

0

### Komo probs

In [56]:
way0 = C.addFrame('way0', 'start_area')

way0.setShape(ry.ST.marker, size=[.1])
way0.setRelativePose('t(0.2 0.15 0.1) d(90 0 0 0)')
C.view()

0

In [57]:
def create_grasp_komo(C, gripper_name, grasp_frame_name, q0, qHome):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) 
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)

    komo.addObjective([1., 3.], ry.FS.positionRel, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [-0.03, 0, 0.01])  

    komo.addObjective([1., 3.], ry.FS.scalarProductXX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [1]) 
    komo.addObjective([1., 3.], ry.FS.scalarProductYY, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [1]) 
    komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [1]) 

    komo.addObjective([1., 3.], ry.FS.scalarProductXZ, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 
    # komo.addObjective([1., 3.], ry.FS.scalarProductYY, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [-1]) 


    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret)
    return komo

In [58]:
def throw_position_komo(C, gripper_name, grasp_frame_name):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([1.], ry.FS.positionDiff, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0, 0, 0])
    komo.addObjective([3.], ry.FS.scalarProductXX, [gripper_name, 'way0'], ry.OT.eq, [1e1], [0])
    ret2 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret2)
    return komo

In [59]:
q0 = C.getJointState()
komo = create_grasp_komo(C, 'l_gripper', 'cargo', q0, qHome)
path = komo.getPath()

komo2 = throw_position_komo(C, 'l_gripper', 'cargo')
path2 = komo2.getPath()

{ time: 0.003065, evals: 14, done: 1, feasible: 1, sos: 0.1319, f: 0, ineq: 0, eq: 0.00513491 }
====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):176.816 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    138.063  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    99.4683  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    62.5688  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    28.9749  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):     9.3213  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):    3.19908  ACCEPT
--newton-- it:   7  |Delta|:        0.2  alpha:          1  evals:   8  f(y):   0.642644  ACCEPT
--newton-- it:   8  |Delta|:   0.143353  alpha:          1  evals:   9  f(y)

### Pick up ball

In [60]:
bot = ry.BotOp(C, useRealRobot=False)
bot.home(C)

bot.move(path, [3.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

bot.gripperCloseGrasp(ry._left, 'cargo') 
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

bot.move(path2, [3.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

# bot.gripperMove(ry._left)
# while not bot.gripperDone(ry._left):
#     bot.sync(C, .1)

pre_throw_pos = C.getFrame('l_gripper').getPosition()


bot.home(C)

-- kin_physx.cpp:addJoint:298(0) ADDING JOINT l_panda_joint7-cargo of type rigid with rel [0, 0, 0]


### Calculate throw

In [61]:
import numpy as np
from scipy.optimize import minimize

def calculate_throw_parameters_bin(initial_position, bin_dimensions, gravity=9.81):
    def loss_function(params):
        release_height, v_magnitude, theta_xz = params
        x0, y0, z0 = initial_position
        bottom_left = bin_dimensions['bottom_left']
        top_right = bin_dimensions['top_right']
        
        release_position = (x0, y0, release_height)
        
        v_x = v_magnitude * np.cos(theta_xz)
        v_z = v_magnitude * np.sin(theta_xz)
        
        t_flight = 2 * v_z / gravity 
        
        x_landing = x0 + v_x * t_flight
        
        within_x_bounds = bottom_left[0] <= x_landing <= top_right[0]
        within_y_bounds = bottom_left[1] <= y0 <= top_right[1]
        
        if within_x_bounds and within_y_bounds:
            return 0  
        else:
            dx = max(0, bottom_left[0] - x_landing, x_landing - top_right[0])
            dy = max(0, bottom_left[1] - y0, y0 - top_right[1])
            return dx + dy 

    initial_guess = [initial_position[2], 5.0, np.pi / 4] 
    
    bounds = [(initial_position[2], initial_position[2] + 1.0), 
              (1.0, 20.0),  
              (0, np.pi / 2)]  
    
    result = minimize(loss_function, initial_guess, bounds=bounds, method='SLSQP')
    
    if result.success:
        release_height, v_magnitude, theta_xz = result.x
        v_x = v_magnitude * np.cos(theta_xz)
        v_z = v_magnitude * np.sin(theta_xz)
        return {
            'release_position': (initial_position[0], initial_position[1], release_height),
            'velocity': (v_x, 0, v_z),
            'angle_xz': theta_xz,
        }
    else:
        raise ValueError("Optimization failed to find valid parameters.")

initial_pos = C.getFrame('l_gripper').getPosition()
print("Initial Position:", initial_pos)
bin_dims = {
    'bottom_left': (-0.25, -0.75, 0.14), 
    'top_right': (0.25, -0.25, 0.24)  
}
# result = calculate_throw_parameters_bin(initial_pos, bin_dims)
result = calculate_throw_parameters_bin(pre_throw_pos, bin_dims)

print("Release Position:", result['release_position'])
print("Velocity:", result['velocity'])
print("Release Angle (radians):", result['angle_xz'])


Initial Position: [0.47567483 1.99561983 0.67380133]
Release Position: (0.2744370668676769, 1.8231970429722304, 0.9920656427971468)
Velocity: (0.7071044642017735, 0, 0.7071090981637301)
Release Angle (radians): 0.7854014401033715


In [62]:
# Add frame for the release position
release_frame = C.addFrame('release')
release_frame.setShape(ry.ST.marker, size=[.5])
release_frame.setPosition(result['release_position'])
C.view()

0

In [63]:
del C
del bot

-- bot.cpp:~BotOp:135(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:57(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation
